In [5]:
!pip install yfinance yoptions yahoo_fin py_vollib
from IPython.core.display import clear_output
clear_output()

In [6]:
# Set option to display all columns
pd.set_option('display.max_columns', None)

# Optionally, set the max rows displayed in the output as well
pd.set_option('display.max_rows', 100)

In [21]:
import os

def find_file_path(filename, search_directory):
    """
    Search for the given filename within the specified directory and its subdirectories.
    Returns the absolute path if the file is found, or None if it's not found.

    :param filename: The name of the file to search for.
    :param search_directory: The directory to start the search from.
    :return: The absolute path to the file or None.
    """
    for root, dirs, files in os.walk(search_directory):
        if filename in files:
            return os.path.abspath(os.path.join(root, filename))
    return None

# Use the function to find the path to 'QTA-lib.py'
file_path = find_file_path('QTA-lib.py', '../TA/')

if file_path:
    print(f"File found at: {file_path}")
else:
    print("File not found.")


File found at: c:\Users\Administrator\Desktop\DataSciencePortfolio\QuantitativeFinance\TA\QTA-lib.py


# Data Retrieval: 
Fetch historical data for SPY.

In [25]:
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import numpy as np

# Fundemental & Technical Data
import yfinance as yf 

# Options Data
import yoptions as yo
from yahoo_fin import options

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import requests


# Import the custom QTA library
import sys
sys.path.append('~\TA\')

from QTA_lib import fetch_technical_data


SyntaxError: unterminated string literal (detected at line 19) (502689568.py, line 19)

In [19]:
def fetch_and_process_data(_asset):
    # Fetch data for the specified asset
    hist = yf.download(_asset, start='2022-01-01') 

    # Indicator calculations as defined earlier
    def bollinger_bands(data, window=20, num_std=2):
        rolling_mean = data['Close'].rolling(window=window).mean()
        rolling_std = data['Close'].rolling(window=window).std()
        data['Bollinger_High'] = rolling_mean + (rolling_std * num_std)
        data['Bollinger_Low'] = rolling_mean - (rolling_std * num_std)
        return data

    def macd(data, short_window=12, long_window=26, signal_window=9):
        short_ema = data['Close'].ewm(span=short_window, adjust=False).mean()
        long_ema = data['Close'].ewm(span=long_window, adjust=False).mean()
        data['MACD'] = short_ema - long_ema
        data['Signal'] = data['MACD'].ewm(span=signal_window, adjust=False).mean()
        return data

    def rsi(data, periods=14, ema=True):
        close_delta = data['Close'].diff()
        up = close_delta.clip(lower=0)
        down = -1 * close_delta.clip(upper=0)
        
        if ema:
            ma_up = up.ewm(com=periods - 1, adjust=True, min_periods=periods).mean()
            ma_down = down.ewm(com=periods - 1, adjust=True, min_periods=periods).mean()
        else:
            ma_up = up.rolling(window=periods, adjust=False).mean()
            ma_down = down.rolling(window=periods, adjust=False).mean()
        
        rsi = ma_up / ma_down
        data['RSI'] = 100 - (100 / (1 + rsi))
        return data
        
    def obv(data):
        """Calculate On-Balance Volume."""
        obv = (np.sign(data['Close'].diff()) * data['Volume']).fillna(0).cumsum()
        data['OBV'] = obv
        return data

    def atr(data, window=14):
        """Calculate Average True Range (ATR)."""
        high_low = data['High'] - data['Low']
        high_close = np.abs(data['High'] - data['Close'].shift())
        low_close = np.abs(data['Low'] - data['Close'].shift())
        ranges = pd.concat([high_low, high_close, low_close], axis=1)
        true_range = np.max(ranges, axis=1)
        data['ATR'] = true_range.rolling(window=window).mean()
        return data

    def woodie_pivots(data):
        # Calculate Woodie's pivot points
        data['Pivot'] = (data['High'] + data['Low'] + 2 * data['Close']) / 4
        data['R1'] = 2 * data['Pivot'] - data['Low']
        data['S1'] = 2 * data['Pivot'] - data['High']
        data['R2'] = data['Pivot'] + (data['High'] - data['Low'])
        data['S2'] = data['Pivot'] - (data['High'] - data['Low'])
        data['R3'] = data['High'] + 2 * (data['Pivot'] - data['Low'])
        data['S3'] = data['Low'] - 2 * (data['High'] - data['Pivot'])
        data['R4'] = data['Pivot'] + 3 * (data['High'] - data['Low'])
        data['S4'] = data['Pivot'] - 3 * (data['High'] - data['Low'])
        return data

    # Apply each indicator function to the data
    hist = bollinger_bands(hist)
    hist = macd(hist)
    hist = rsi(hist)
    hist = woodie_pivots(hist)
    hist = obv(hist)
    hist = atr(hist)
    # Repeat for other indicators as necessary...

    # Note: No explicit parallel processing applied here due to sequential dependency of calculations on data.

    # Ensure all NaN values created by indicators are handled appropriately
    hist.dropna(inplace=True)

    return hist

# Example usage
spy_data = fetch_and_process_data("SPY")
print(spy_data.tail())  # Display the last few rows to verify the outcome
# spy_data

[*********************100%%**********************]  1 of 1 completed

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2024-03-22  522.109985  522.609985  520.969971  521.210022  521.210022   
2024-03-25  519.799988  520.950012  519.609985  519.770020  519.770020   
2024-03-26  521.229980  521.580017  518.400024  518.809998  518.809998   
2024-03-27  521.710022  523.210022  519.489990  523.169983  523.169983   
2024-03-28  523.210022  524.609985  522.780029  523.070007  523.070007   

              Volume  Bollinger_High  Bollinger_Low      MACD    Signal  \
Date                                                                      
2024-03-22  79023000      522.701581     503.012415  5.747561  5.649193   
2024-03-25  48512100      523.295382     503.796616  5.637581  5.646871   
2024-03-26  65463700      523.636494     504.643505  5.410586  5.599614   
2024-03-27  82999800      524.538896     505.432101  5.518887  5.583468   
2024-03-28  96223500      525.3

# Check Data For any errors:

In [9]:
from pandas.tseries.holiday import USFederalHolidayCalendar

def check_data_errors(data):
    errors = []

    # Check for missing values
    if data.isnull().values.any():
        errors.append("Issue: Data contains missing values.")
    
    # Check for duplicate dates
    if data.index.duplicated().any():
        errors.append("Issue: Data contains duplicate dates.")
    
    # Outliers in price data
    z_scores = np.abs((data['Close'] - data['Close'].mean()) / data['Close'].std())
    if z_scores[z_scores > 3].any():
        errors.append("Issue: Data contains potential outliers in 'Close' prices.")
    
    # Volume checks
    if (data['Volume'] == 0).any():
        errors.append("Issue: Data contains days with zero volume.")
    if ((data['Volume'].diff() / data['Volume']).abs() > 5).any():
        errors.append("Issue: Data contains unexpected spikes in volume.")
    
    # Continuity of dates, excluding weekends and public holidays
    cal = USFederalHolidayCalendar()
    holidays = cal.holidays(start=data.index.min(), end=data.index.max())
    business_days = pd.date_range(start=data.index.min(), end=data.index.max(), freq='B')
    business_days = business_days[~business_days.isin(holidays)]  # Exclude holidays
    
    missing_dates = business_days.difference(data.index).tolist()
    if missing_dates:
        formatted_dates = ', '.join([d.strftime('%Y-%m-%d') for d in missing_dates])
        errors.append(f"Issue: Data might be missing trading days: {formatted_dates}")

    return errors

# Example usage
spy_data = fetch_and_process_data("SPY")  # Assuming this function returns data with DateTimeIndex
errors = check_data_errors(spy_data)
if errors:
    for error in errors:
        print(error)
else:
    print("No issues detected in the data.")


[*********************100%%**********************]  1 of 1 completed

Issue: Data might be missing trading days: 2022-04-15, 2023-04-07


The error message indicating missing trading days for specific dates such as April 19, 2019, April 10, 2020, April 2, 2021, April 15, 2022, and April 7, 2023, highlights dates that are actually Good Friday. In the United States, the stock market (NYSE, NASDAQ) is closed on Good Friday, which is not a federal holiday and therefore not included in the USFederalHolidayCalendar. This explains why these dates were flagged as missing trading days by the previous function.

To address this and accurately reflect the trading calendar, we need to manually account for Good Friday and potentially other market-specific closures not covered by the federal holiday calendar. Here's an updated version of the function that checks for missing trading days, now including an adjustment for Good Friday and a more general approach to handling non-trading days:

Frankly, this is good enough for 

# Credit Spread Pricing
Here we want to use R1-4 and S1-S4 to find the nearest 50 cent spread.

We'll then try to determine the probablity of each expiring.

In [10]:
import pandas as pd
import numpy as np

def round_to_nearest_50_cents(value):
    """Round the value to the nearest 50 cents."""
    return np.round(value * 2, 0) / 2

def generate_credit_spreads(data):
    # Initialize a list to hold the spreads for each day
    spreads = []
    
    # Iterate through each row in the DataFrame
    for index, row in data.iterrows():
        # Initialize a dictionary for the current day's spreads
        day_spreads = {
            'Date': index,
            'R1-R2 Put Spread': None,
            'R1-R3 Put Spread': None,
            'R1-R4 Put Spread': None,
            'S1-S2 Call Spread': None,
            'S1-S3 Call Spread': None,
            'S1-S4 Call Spread': None,
        }
        
        # Calculate the put credit spreads for each pair of resistances
        for i in range(2, 5):
            sell_strike = round_to_nearest_50_cents(row['R1'])
            buy_strike = round_to_nearest_50_cents(row[f'R{i}'])
            day_spreads[f'R1-R{i} Put Spread'] = (sell_strike, buy_strike)
        
        # Calculate the call credit spreads for each pair of supports
        for i in range(2, 5):
            sell_strike = round_to_nearest_50_cents(row['S1'])
            buy_strike = round_to_nearest_50_cents(row[f'S{i}'])
            day_spreads[f'S1-S{i} Call Spread'] = (sell_strike, buy_strike)
        
        # Add the current day's spreads to the list
        spreads.append(day_spreads)
    
    # Convert the list of spreads into a DataFrame for easier viewing
    spreads_df = pd.DataFrame(spreads)
    spreads_df.set_index('Date', inplace=True)
    return spreads_df

# Assuming spy_data is already defined and contains the necessary columns
# Generate the credit spreads
credit_spreads = generate_credit_spreads(spy_data)

credit_spreads

,R1-R2 Put Spread,R1-R3 Put Spread,R1-R4 Put Spread,S1-S2 Call Spread,S1-S3 Call Spread,S1-S4 Call Spread
Date,,,,,,
2022-01-31,"(455.0, 458.0)","(455.0, 465.5)","(455.0, 479.0)","(444.5, 437.0)","(444.5, 434.0)","(444.5, 416.0)"
2022-02-01,"(456.5, 458.5)","(456.5, 463.0)","(456.5, 471.5)","(449.5, 445.0)","(449.5, 443.0)","(449.5, 431.5)"
2022-02-02,"(460.0, 461.5)","(460.0, 465.0)","(460.0, 471.5)","(455.0, 451.5)","(455.0, 449.5)","(455.0, 441.5)"
2022-02-03,"(450.0, 455.0)","(450.0, 457.5)","(450.0, 470.0)","(443.0, 440.5)","(443.0, 435.5)","(443.0, 426.0)"
2022-02-04,"(453.0, 457.5)","(453.0, 462.0)","(453.0, 475.5)","(444.0, 439.5)","(444.0, 435.5)","(444.0, 421.5)"
...,...,...,...,...,...,...
2024-03-22,"(522.0, 523.0)","(522.0, 523.5)","(522.0, 526.5)","(520.5, 520.0)","(520.5, 519.0)","(520.5, 516.5)"
2024-03-25,"(520.5, 521.5)","(520.5, 522.0)","(520.5, 524.0)","(519.0, 518.5)","(519.0, 518.0)","(519.0, 516.0)"
2024-03-26,"(520.5, 522.5)","(520.5, 523.5)","(520.5, 529.0)","(517.0, 516.0)","(517.0, 514.0)","(517.0, 510.0)"


In [11]:
# POssible Rewrite of the function....

In [12]:

def round_to_nearest_50_cents(value):
    """Round the value to the nearest 50 cents."""
    return np.round(value * 2) / 2

def generate_credit_spreads(data):
    """
    Generate a DataFrame with credit spreads for each day.

    Parameters:
    - data: DataFrame containing the input data with columns 'R1', 'R2', 'R3', 'R4', 'S1', 'S2', 'S3', 'S4'

    Returns:
    - DataFrame with credit spreads for put and call options
    """
    # Initialize a list to hold the DataFrame for each day's credit spreads
    spreads_list = []

    # Iterate through each row in the DataFrame
    for index, row in data.iterrows():
        # Calculate the put credit spreads for each pair of resistances
        r1_r2_put_spread = (round_to_nearest_50_cents(row['R1']), round_to_nearest_50_cents(row['R2']))
        r1_r3_put_spread = (round_to_nearest_50_cents(row['R1']), round_to_nearest_50_cents(row['R3']))
        r1_r4_put_spread = (round_to_nearest_50_cents(row['R1']), round_to_nearest_50_cents(row['R4']))
        
        # Calculate the call credit spreads for each pair of supports
        s1_s2_call_spread = (round_to_nearest_50_cents(row['S2']), round_to_nearest_50_cents(row['S1']))
        s1_s3_call_spread = (round_to_nearest_50_cents(row['S3']), round_to_nearest_50_cents(row['S1']))
        s1_s4_call_spread = (round_to_nearest_50_cents(row['S4']), round_to_nearest_50_cents(row['S1']))

        # Store the calculated spreads in a dictionary
        day_spreads = {
            'Date': index,
            'R1-R2 Put Spread': r1_r2_put_spread,
            'R1-R3 Put Spread': r1_r3_put_spread,
            'R1-R4 Put Spread': r1_r4_put_spread,
            'S1-S2 Call Spread': s1_s2_call_spread,
            'S1-S3 Call Spread': s1_s3_call_spread,
            'S1-S4 Call Spread': s1_s4_call_spread,
        }

        # Append the current day's spreads to the list
        spreads_list.append(day_spreads)

    # Convert the list of spreads into a DataFrame for easier viewing
    spreads_df = pd.DataFrame(spreads_list)
    spreads_df.set_index('Date', inplace=True)
    return spreads_df

# Assuming data is already defined and contains the necessary columns
# Example usage:
# data = pd.read_csv('path_to_data.csv') # Replace with actual path to data
credit_spreads = generate_credit_spreads(spy_data)
print(credit_spreads)


           R1-R2 Put Spread R1-R3 Put Spread R1-R4 Put Spread  \
Date                                                            
2022-01-31   (455.0, 458.0)   (455.0, 465.5)   (455.0, 479.0)   
2022-02-01   (456.5, 458.5)   (456.5, 463.0)   (456.5, 471.5)   
2022-02-02   (460.0, 461.5)   (460.0, 465.0)   (460.0, 471.5)   
2022-02-03   (450.0, 455.0)   (450.0, 457.5)   (450.0, 470.0)   
2022-02-04   (453.0, 457.5)   (453.0, 462.0)   (453.0, 475.5)   
...                     ...              ...              ...   
2024-03-22   (522.0, 523.0)   (522.0, 523.5)   (522.0, 526.5)   
2024-03-25   (520.5, 521.5)   (520.5, 522.0)   (520.5, 524.0)   
2024-03-26   (520.5, 522.5)   (520.5, 523.5)   (520.5, 529.0)   
2024-03-27   (525.0, 526.0)   (525.0, 529.0)   (525.0, 533.5)   
2024-03-28   (524.0, 525.0)   (524.0, 526.0)   (524.0, 529.0)   

           S1-S2 Call Spread S1-S3 Call Spread S1-S4 Call Spread  
Date                                                              
2022-01-31    (437.0

#  Option Spread Probability Estimation Using Black-Scholes Model
Here we introduce a method to estimate the probabilities of credit spreads expiring worthless using the Black-Scholes option pricing model. This approach is valuable for traders who use options as a financial instrument for hedging or speculative purposes.

The `calculate_option_probabilities` function estimates the likelihood of options spreads expiring worthless. This function requires several inputs:

- spy_data: The DataFrame containing 'Close' prices of the stock.
- credit_spreads: The DataFrame containing the strike prices for the spreads.
- volatility: The annualized volatility of the stock.
- risk_free_rate: The annual risk-free interest rate.
- time_to_expiration: The time to expiration of the options in year}

Inside the function, we define a nested helper function `calculate_cdf`, which calculates the CDF for the Black-Scholes model given a strike price and the current stock price.





In [13]:
import pandas as pd
import numpy as np
from scipy.stats import norm
print(spy_data)
print(credit_spreads)
def calculate_option_probabilities(spy_data, credit_spreads, volatility, risk_free_rate, time_to_expiration):
    """
    Calculate the probabilities of credit spreads expiring worthless using the Black-Scholes model.
    
    Parameters:
    - spy_data: DataFrame containing 'Close' prices of the stock.
    - credit_spreads: DataFrame containing the strike prices for the spreads.
    - volatility: Annualized volatility of the stock.
    - risk_free_rate: Annual risk-free interest rate.
    - time_to_expiration: Time to expiration of the options in years.
    
    Returns:
    - DataFrame with probabilities of each spread expiring worthless.
    """
    def calculate_cdf(strike, current_price, volatility, time_to_expiration, risk_free_rate):
        """Calculate the cumulative distribution function for Black-Scholes."""
        d1 = (np.log(current_price / strike) + (risk_free_rate + 0.5 * volatility ** 2) * time_to_expiration) / (volatility * np.sqrt(time_to_expiration))
        return norm.cdf(d1)
    
    probabilities = []
    
    for index, row in spy_data.iterrows():
        current_price = row['Close']
        
        # Adjusted to directly use 'R' and 'S' values from `credit_spreads`
        for i in range(1, 5):
            if f'R{i}' in credit_spreads.columns and f'S{i}' in credit_spreads.columns:
                # For put spreads, using R values
                sell_strike_put = row[f'R{i}']
                buy_strike_put = row[f'R{i}'] - 1  # Example adjustment, customize as needed
                prob_put = calculate_cdf(sell_strike_put, current_price, volatility, time_to_expiration, risk_free_rate)
                
                # For call spreads, using S values
                sell_strike_call = row[f'S{i}']
                buy_strike_call = row[f'S{i}'] + 1  # Example adjustment, customize as needed
                prob_call = 1 - calculate_cdf(buy_strike_call, current_price, volatility, time_to_expiration, risk_free_rate)
                
                probabilities.append({
                    'Date': index,
                    f'R{i} Put Spread Probability': prob_put,
                    f'S{i} Call Spread Probability': prob_call
                })
    
    probabilities_df = pd.DataFrame(probabilities).set_index('Date')
    return probabilities_df

# Assuming you have calculated or have the values for the following variables:
volatility = 0.2  # Example volatility
risk_free_rate = 0.01  # Example risk-free rate
time_to_expiration = 1/52  # 1 week to expiration

# Calculate probabilities
probabilities_df = calculate_option_probabilities(spy_data, spy_data, volatility, risk_free_rate, time_to_expiration)
print(probabilities_df)

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2022-01-31  441.239990  450.279999  439.809998  449.910004  434.748627   
2022-02-01  450.679993  453.630005  446.940002  452.950012  437.686188   
2022-02-02  455.500000  458.119995  453.049988  457.350006  441.937866   
2022-02-03  450.950012  452.970001  445.709991  446.600006  431.550140   
2022-02-04  446.350006  452.779999  443.829987  448.700012  433.579407   
...                ...         ...         ...         ...         ...   
2024-03-22  522.109985  522.609985  520.969971  521.210022  521.210022   
2024-03-25  519.799988  520.950012  519.609985  519.770020  519.770020   
2024-03-26  521.229980  521.580017  518.400024  518.809998  518.809998   
2024-03-27  521.710022  523.210022  519.489990  523.169983  523.169983   
2024-03-28  523.210022  524.609985  522.780029  523.070007  523.070007   

               Volume  Bollinger_High

## Get This weeks Spread Probabilities

In [14]:
# Assuming probabilities_df is indexed by date
specific_date_probabilities = probabilities_df.loc['2024-02-20']

print(specific_date_probabilities)


            R1 Put Spread Probability  S1 Call Spread Probability  \
Date                                                                
2024-02-20                   0.451202                    0.463349   
2024-02-20                        NaN                         NaN   
2024-02-20                        NaN                         NaN   
2024-02-20                        NaN                         NaN   

            R2 Put Spread Probability  S2 Call Spread Probability  \
Date                                                                
2024-02-20                        NaN                         NaN   
2024-02-20                   0.399903                    0.401938   
2024-02-20                        NaN                         NaN   
2024-02-20                        NaN                         NaN   

            R3 Put Spread Probability  S3 Call Spread Probability  \
Date                                                                
2024-02-20                      

## Get the Probabilites greater than 50%

In [15]:
# Filter for probabilities greater than 60%
filtered_probabilities = probabilities_df[probabilities_df > 0.5]

# The operation above will leave NaNs where the condition is not met, so you might want to drop these for clarity
filtered_probabilities.dropna(how='all', inplace=True)

# To drop columns where all values are NaN after filtering (in case you're interested in dates with any probability > 60%)
filtered_probabilities.dropna(axis=1, how='all', inplace=True)

print(filtered_probabilities)


            S1 Call Spread Probability
Date                                  
2022-11-25                    0.502959
2023-05-09                    0.502199
2023-07-03                    0.500612
2023-11-24                    0.511117
2023-11-27                    0.501035
2023-12-28                    0.502410
2024-03-25                    0.500831


# Binomial Tree Model for Credit Spread Probability Estimation

In the previous sections, we gathered stock data using `fetch_and_process_data` function and generated credit spreads with `generate_credit_spreads` function. With these preparations in place, we are now ready to apply a Binomial Tree model to estimate the probability of weekly credit spreads expiring worthless.

The Binomial Tree model is a discrete-time model for valuing options, which is particularly useful for American options. This model represents the evolution of the option's key underlying variable via a binomial lattice (tree), for a given number of time steps between the valuation and expiration dates.



### Constructing the Binomial Tree

In [16]:
# Function to calculate the up and down factors for the Binomial Tree model
def calculate_up_down_factors(volatility, time_to_expiration, steps):
    # Calculate the up and down factors
    up = np.exp(volatility * np.sqrt(time_to_expiration / steps))
    down = 1 / up
    return up, down

# Define the number of steps in the Binomial Tree
steps = int(time_to_expiration * 52)  # Number of weeks in a year

# # Fetch the necessary data from the previously defined functions
# spy_data = fetch_and_process_data('SPY')
# credit_spreads = generate_credit_spreads(spy_data)

# Calculate the up and down factors
up, down = calculate_up_down_factors(volatility, time_to_expiration, steps)

def construct_binomial_tree(spy_data, credit_spreads, volatility, risk_free_rate, steps):
    up, down = calculate_up_down_factors(volatility, 1/52, steps)  # Example: 1 week to expiration
    current_price = spy_data['Close'].iloc[-1]  # Assuming we use the latest close price as current

    # Initialize binomial tree
    tree = np.zeros([steps + 1, steps + 1])
    
    # Calculate stock price at each node
    for i in range(steps + 1):
        for j in range(i + 1):
            tree[j, i] = current_price * (up ** (i - j)) * (down ** j)
    
    # More code will go here to calculate payoffs and probabilities
    return tree

# Construct the binomial tree
binomial_tree = construct_binomial_tree(spy_data, credit_spreads, volatility, risk_free_rate, steps)
binomial_tree

array([[523.07000732, 537.78041277],
       [  0.        , 508.76198922]])

## Calculate Option Payoffs: 
At each final node (at expiration), calculate the payoff of the put and call spreads based on their strike prices. For a credit spread, the payoff is typically the difference between the premiums of the sold and bought options if the spread expires in the money.

In [17]:
# GPT DEBUGGING CODE
# spy_data = fetch_and_process_data("SPY")
# print(spy_data.tail(1)) 
# print(credit_spreads.tail(1))

def calculate_binomial_tree_probabilities(stock_data, credit_spreads_data, volatility, risk_free_rate):
    # Fetch the latest stock price
    current_price = stock_data['Close'].iloc[-1]
    
    # Calculate the up and down factors
    time_to_expiration = 1 / 52  # One week to expiration
    up_factor = np.exp(volatility * np.sqrt(time_to_expiration))
    down_factor = 1 / up_factor
    steps = int(time_to_expiration * 52)  # Assuming weekly steps in a year
    
    # Initialize the binomial tree structure
    binomial_tree = np.zeros((steps + 1, steps + 1))
    for i in range(steps + 1):
        for j in range(i + 1):
            binomial_tree[j, i] = current_price * (up_factor ** (i - j)) * (down_factor ** j)
    
    # Calculate payoffs for each node
    for spread_row in credit_spreads_data.itertuples():
        for i in range(steps, -1, -1):
            for j in range(i + 1):
                stock_price = binomial_tree[j, i]
                # Calculate payoffs for put and call spreads
                put_payoff = max(0, spread_row.R1 - stock_price)
                call_payoff = max(0, stock_price - spread_row.S1)
                # Discount the payoff back to the present value
                if i < steps:  # Not at the terminal node
                    put_payoff = (binomial_tree[j, i + 1] * put_payoff + binomial_tree[j + 1, i + 1] * put_payoff) / (2 * np.exp(risk_free_rate * time_to_expiration))
                    call_payoff = (binomial_tree[j, i + 1] * call_payoff + binomial_tree[j + 1, i + 1] * call_payoff) / (2 * np.exp(risk_free_rate * time_to_expiration))
                # Set the payoff in the tree
                binomial_tree[j, i] = put_payoff + call_payoff
    
    # Sum the probabilities of the nodes where the spread is worthless
    probability_spread_worthless = sum(binomial_tree[:, 0] == 0) / len(binomial_tree[:, 0])
    return probability_spread_worthless

# Fetch the necessary data
spy_data = fetch_and_process_data('SPY')
credit_spreads = generate_credit_spreads(spy_data)

# Calculate the probability
probability = calculate_binomial_tree_probabilities(spy_data, credit_spreads, 0.2, 0.01)
print(f"Probability of weekly credit spreads expiring worthless: {probability}")


[*********************100%%**********************]  1 of 1 completed

AttributeError: 'Pandas' object has no attribute 'R1'

## Price the Options Using the Tree: 
Starting from the terminal nodes, work backward through the tree to calculate the present value of the option payoffs at each node. This involves discounting future payoffs by the risk-free rate and accounting for the probability of up and down movements.

## Estimate Probabilities of Expiring Worthless: 

The goal is to estimate the probabilities that the credit spreads will expire worthless. To do this, you need to assess the likelihood of the underlying asset's price being beyond the strike prices of the spreads at expiration.